In [1]:
import pandas as pd

In [2]:
#we take in the data for the current european elections
current_europe_election=pd.read_csv("../data/kerg.csv",skiprows=2, sep=';',dtype=str)
#but only when it is on Kreis-level of administration
current_europe_election=current_europe_election[(current_europe_election["Nr"].str.len()>4)]

In [3]:
columnarray=[]
for columnname in current_europe_election.columns:
#we rephrase the name of the columns by going through the columnnames    
    if("Unnamed" in columnname):
        columnarray.append(previousrow+"_2019")
    else:
        columnarray.append(columnname+"_2024")
    previousrow=columnname
current_europe_election.columns=columnarray
current_europe_election=current_europe_election.dropna(subset=["Gebiet_2024"])
current_europe_election

,Nr_2024,Gebiet_2024,gehört zu_2024,Wahlberechtigte_2024,Wahlberechtigte_2019,Wählende_2024,Wählende_2019,Ungültige Stimmen_2024,Ungültige Stimmen_2019,Gültige Stimmen_2024,...,Parlament aufmischen – Stimme der Letzten Generation_2019,Partei der Vernunft_2024,Partei der Vernunft_2019,Partei des Fortschritts_2024,Partei des Fortschritts_2019,"V-Partei³ - Partei für Veränderung, Vegetarier und Veganer_2024","V-Partei³ - Partei für Veränderung, Vegetarier und Veganer_2019",Übrige_2024,Übrige_2019,Unnamed: 82_2019
2,01001,"Flensburg, Stadt",01,70929,70431,41851,39647,299,237,41552,...,NaN,43,NaN,340,NaN,87,NaN,NaN,732,NaN
3,01002,"Kiel, Landeshauptstadt",01,187041,186195,117097,109595,550,577,116547,...,NaN,63,NaN,832,NaN,210,NaN,NaN,1733,NaN
4,01003,"Lübeck, Hansestadt",01,166429,166530,96558,90946,604,461,95954,...,NaN,79,NaN,604,NaN,158,NaN,NaN,1401,NaN
5,01004,"Neumünster, Stadt",01,59983,60215,33019,29036,282,231,32737,...,NaN,24,NaN,313,NaN,53,NaN,NaN,582,NaN
6,01051,Dithmarschen,01,108756,107045,68033,58244,506,475,67527,...,NaN,55,NaN,421,NaN,77,NaN,NaN,1055,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,16073,Saalfeld-Rudolstadt,16,84724,88342,49575,53882,329,1034,49246,...,NaN,38,NaN,172,NaN,41,NaN,NaN,1736,NaN
428,16074,Saale-Holzland-Kreis,16,67684,68278,45600,45101,411,1003,45189,...,NaN,31,NaN,145,NaN,52,NaN,NaN,1500,NaN
429,16075,Saale-Orla-Kreis,16,64882,67029,38507,42514,183,884,38324,...,NaN,27,NaN,98,NaN,29,NaN,NaN,1359,NaN
430,16076,Greiz,16,80024,82940,52514,53197,509,1139,52005,...,NaN,37,NaN,147,NaN,49,NaN,NaN,1833,NaN


In [4]:
#and take only the data for the afd and all valid votes and the district number
current_europe_election_afd=current_europe_election[["Nr_2024","Gebiet_2024","Gültige Stimmen_2024","Alternative für Deutschland_2024"]]
current_europe_election_afd[current_europe_election_afd["Nr_2024"]=="11000"]

,Nr_2024,Gebiet_2024,Gültige Stimmen_2024,Alternative für Deutschland_2024
346,11000,"Berlin, Stadt",1545022,179298


In [5]:
#we get the data for the last german election 
latest_german_election=pd.read_csv("../data/btw2021kreis.csv",skiprows=4, sep=';',dtype=str)
latest_german_election=latest_german_election[(latest_german_election["Statistische Kennziffer"].str.len()>4)]
#and sum the  data for berlin because it was split in east and west

berlin_latest_election=latest_german_election[latest_german_election["Kreisfreie Stadt bzw. Stadtkreis, Landkreis bzw. Kreis"].astype(str).str.contains("Berlin")]
berlin_latest_election[berlin_latest_election.columns[3:]]=berlin_latest_election[berlin_latest_election.columns[3:]].astype(int)
berlin_df_added=pd.DataFrame(berlin_latest_election.sum()).T
berlin_df_added["Land"]="BE"
berlin_df_added["Statistische Kennziffer"]="11000"
berlin_df_added["Kreisfreie Stadt bzw. Stadtkreis, Landkreis bzw. Kreis"]="Berlin"
berlin_df_added

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/3630396570.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  berlin_latest_election[berlin_latest_election.columns[3:]]=berlin_latest_election[berlin_latest_election.columns[3:]].astype(int)


,Land,Statistische Kennziffer,"Kreisfreie Stadt bzw. Stadtkreis, Landkreis bzw. Kreis",Wahlberechtigte,Wähler,Ungültige,Gültige,CDU,SPD,AfD,...,III. Weg.1,Bündnis21.1,LIEBE.1,LKR.1,PdF,LfK,SSW.1,Team Todenhöfer.1,UNABHÄNGIGE.1,Volt.1
0,BE,11000,Berlin,2468919,1856903,31631,1825272,344838,417163,147134,...,0,0,0,1232,0,0,0,20168,0,11645


In [7]:
latest_german_election_columns=[]
#again some columnname cleaning
for column in latest_german_election.columns:
    column=column.replace(".1","_Zweitstimme")
    latest_german_election_columns.append(column+"_2021")
latest_german_election=pd.concat([latest_german_election,berlin_df_added]).reset_index(drop=True)
latest_german_election.columns=latest_german_election_columns
#and we take in the data for the Afd an the list votes
latest_german_election_afd=latest_german_election[["Statistische Kennziffer_2021","Gültige_Zweitstimme_2021",'AfD_Zweitstimme_2021']]
latest_german_election_afd=latest_german_election_afd.dropna(subset=["Statistische Kennziffer_2021"])
latest_german_election_afd
latest_german_election_afd

ValueError: Length mismatch: Expected axis has 188 elements, new values have 94 elements

In [8]:
#we take the data for 2019s european election
european_election_2019=pd.read_csv("../data/ew19_kerg.csv",skiprows=2, sep=';',dtype=str)
european_election_2019=european_election_2019[(european_election_2019["Nr"].str.len()>4)]

columnarray=[]
for columnname in european_election_2019.columns:
    
    if("Unnamed" in columnname):
        columnarray.append(previousrow+"_2014")
    else:
        columnarray.append(columnname+"_2019")
    previousrow=columnname
european_election_2019.columns=columnarray
european_election_2019=european_election_2019.dropna(subset=["Gebiet_2019"])
european_election_2019_afd=european_election_2019[["Nr_2019","Gültige_2019","Alternative für Deutschland_2019","Gültige_2014","Alternative für Deutschland_2014"]]
european_election_2019_afd[european_election_2019_afd["Nr_2019"]=="11000"]

,Nr_2019,Gültige_2019,Alternative für Deutschland_2019,Gültige_2014,Alternative für Deutschland_2014
346,11000,1507357,149177,1159666,91759


In [9]:
#we take the data for 2017 german election

german_election_2017=pd.read_csv("../data/btw2017kreis.csv",skiprows=4, sep=';',dtype=str,encoding="latin")
german_election_2017=german_election_2017[(german_election_2017["Statistische Kennziffer"].str.len()>4)]

berlin_2017_election=german_election_2017[german_election_2017["Kreisfreie Stadt bzw. Stadtkreis, Landkreis bzw. Kreis"].astype(str).str.contains("Berlin")]
berlin_2017_election[berlin_2017_election.columns[3:]]=berlin_2017_election[berlin_2017_election.columns[3:]].astype(int)
berlin_2017_election=pd.DataFrame(berlin_2017_election.sum()).T
berlin_2017_election["Land"]="BE"
berlin_2017_election["Statistische Kennziffer"]="11000"
berlin_2017_election["Kreisfreie Stadt bzw. Stadtkreis, Landkreis bzw. Kreis"]="Berlin"
berlin_2017_election
german_election_2017_columns=[]
for column in german_election_2017.columns:
    column=column.replace(".1","_Zweitstimme")
    german_election_2017_columns.append(column+"_2017")
german_election_2017=pd.concat([german_election_2017,berlin_2017_election]).reset_index(drop=True)

german_election_2017.columns=german_election_2017_columns
german_election_2017_afd=german_election_2017[["Statistische Kennziffer_2017","Gültige_Zweitstimme_2017",'AfD_Zweitstimme_2017']]
german_election_2017_afd=german_election_2017_afd.dropna(subset=["Statistische Kennziffer_2017"])


/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/3159791709.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  berlin_2017_election[berlin_2017_election.columns[3:]]=berlin_2017_election[berlin_2017_election.columns[3:]].astype(int)


In [10]:
german_election_2017_afd[german_election_2017_afd["Statistische Kennziffer_2017"]=="11000"]

,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017
402,11000,1869042,225170


In [13]:
# we take the data for the german election from 2012
german_election_2013=pd.read_csv("../data/btw2013kreis.csv",skiprows=4, sep=';',dtype=str,encoding="latin")
german_election_2013=german_election_2013[(german_election_2013["Statistische Kennziffer"].str.len()>4)]
german_election_2013[german_election_2013["Statistische Kennziffer"]=="11000"]
german_election_2013_columns=[]
for column in german_election_2013.columns:
    column=column.replace(".1","_Zweitstimme")
    german_election_2013_columns.append(column+"_2013")

german_election_2013.columns=german_election_2013_columns
german_election_2013_afd=german_election_2013[["Statistische Kennziffer_2013","Gültige_Zweitstimme_2013",'AfD_Zweitstimme_2013']]
german_election_2013_afd=german_election_2013_afd.dropna(subset=["Statistische Kennziffer_2013"])
german_election_2013_afd[german_election_2013_afd["Statistische Kennziffer_2013"]=="11000"]

,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013
326,11000,1787721,88060


In [14]:
#and we sub in the old kreis number for goettingen
german_election_2013_afd["Statistische Kennziffer_2013"]=german_election_2013_afd["Statistische Kennziffer_2013"].str.replace("03152","03159")

In [15]:
#we merge the german election data from 2013 and 2017
german_elections_2013_2017=pd.merge(german_election_2013_afd,german_election_2017_afd,left_on="Statistische Kennziffer_2013",right_on="Statistische Kennziffer_2017", how="outer")
german_elections_2013_2017[german_elections_2013_2017["Statistische Kennziffer_2013"]=="11000"]

,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017
325,11000,1787721,88060,11000,1869042,225170


In [16]:
#and then from 2021
german_elections_2013_2017_2021=pd.merge(german_elections_2013_2017,latest_german_election_afd,left_on="Statistische Kennziffer_2017",right_on="Statistische Kennziffer_2021", how="outer")
german_elections_2013_2017_2021[german_elections_2013_2017_2021["Statistische Kennziffer_2013"]=="11000"]

,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017,Statistische Kennziffer_2021,Gültige_Zweitstimme_2021,AfD_Zweitstimme_2021
324,11000,1787721,88060,11000,1869042,225170,11000,1826581,153694


In [17]:
#we merge the european election data from 2024 and 2019
europe_elections=pd.merge(current_europe_election_afd,european_election_2019_afd,left_on="Nr_2024",right_on="Nr_2019",how="outer")
europe_elections

,Nr_2024,Gebiet_2024,Gültige Stimmen_2024,Alternative für Deutschland_2024,Nr_2019,Gültige_2019,Alternative für Deutschland_2019,Gültige_2014,Alternative für Deutschland_2014
0,01001,"Flensburg, Stadt",41552,4273,01001,39410,2477,25174,1583
1,01002,"Kiel, Landeshauptstadt",116547,9351,01002,109018,6459,75336,4418
2,01003,"Lübeck, Hansestadt",95954,10290,01003,90485,6773,62509,4334
3,01004,"Neumünster, Stadt",32737,5697,01004,28805,2761,27250,1960
4,01051,Dithmarschen,67527,11005,01051,57769,5121,42350,2325
...,...,...,...,...,...,...,...,...,...
396,16073,Saalfeld-Rudolstadt,49246,17307,16073,52848,14592,46150,4107
397,16074,Saale-Holzland-Kreis,45189,14404,16074,44098,10210,40150,3920
398,16075,Saale-Orla-Kreis,38324,14426,16075,41630,11264,37348,3114
399,16076,Greiz,52005,18399,16076,52058,13297,47189,4559


In [18]:
europe_elections[europe_elections["Gebiet_2024"].astype(str).str.contains("Berlin")]

,Nr_2024,Gebiet_2024,Gültige Stimmen_2024,Alternative für Deutschland_2024,Nr_2019,Gültige_2019,Alternative für Deutschland_2019,Gültige_2014,Alternative für Deutschland_2014
324,11000,"Berlin, Stadt",1545022,179298,11000,1507357,149177,1159666,91759


In [19]:
#then we filter only for the relevant columns to calculate percentage splits
afd_europe_elections=europe_elections[["Nr_2024","Gebiet_2024","Alternative für Deutschland_2014","Gültige_2014","Alternative für Deutschland_2019","Gültige_2019","Alternative für Deutschland_2024","Gültige Stimmen_2024"]]
afd_europe_elections

,Nr_2024,Gebiet_2024,Alternative für Deutschland_2014,Gültige_2014,Alternative für Deutschland_2019,Gültige_2019,Alternative für Deutschland_2024,Gültige Stimmen_2024
0,01001,"Flensburg, Stadt",1583,25174,2477,39410,4273,41552
1,01002,"Kiel, Landeshauptstadt",4418,75336,6459,109018,9351,116547
2,01003,"Lübeck, Hansestadt",4334,62509,6773,90485,10290,95954
3,01004,"Neumünster, Stadt",1960,27250,2761,28805,5697,32737
4,01051,Dithmarschen,2325,42350,5121,57769,11005,67527
...,...,...,...,...,...,...,...,...
396,16073,Saalfeld-Rudolstadt,4107,46150,14592,52848,17307,49246
397,16074,Saale-Holzland-Kreis,3920,40150,10210,44098,14404,45189
398,16075,Saale-Orla-Kreis,3114,37348,11264,41630,14426,38324
399,16076,Greiz,4559,47189,13297,52058,18399,52005


In [20]:
#and then we merge the data from german and european elections
europe_german_afd=pd.merge(afd_europe_elections,german_elections_2013_2017_2021,right_on="Statistische Kennziffer_2021",left_on="Nr_2024",how="outer")
europe_german_afd

,Nr_2024,Gebiet_2024,Alternative für Deutschland_2014,Gültige_2014,Alternative für Deutschland_2019,Gültige_2019,Alternative für Deutschland_2024,Gültige Stimmen_2024,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017,Statistische Kennziffer_2021,Gültige_Zweitstimme_2021,AfD_Zweitstimme_2021
0,01001,"Flensburg, Stadt",1583,25174,2477,39410,4273,41552,01001,46265,1855,01001,49271,3702,01001,50818,2756
1,01002,"Kiel, Landeshauptstadt",4418,75336,6459,109018,9351,116547,01002,130686,4778,01002,137368,9524,01002,139961,7023
2,01003,"Lübeck, Hansestadt",4334,62509,6773,90485,10290,95954,01003,112029,4792,01003,117879,10461,01003,117674,7807
3,01004,"Neumünster, Stadt",1960,27250,2761,28805,5697,32737,01004,39388,1965,01004,39861,4124,01004,39971,3751
4,01051,Dithmarschen,2325,42350,5121,57769,11005,67527,01051,73649,3041,01051,77337,6626,01051,80616,7098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,16075,Saale-Orla-Kreis,3114,37348,11264,41630,14426,38324,16075,48979,3527,16075,50721,13452,16075,49552,15071
400,16076,Greiz,4559,47189,13297,52058,18399,52005,16076,61725,4714,16076,63402,16203,16076,61146,16908
401,16077,Altenburger Land,3023,37657,11371,41901,15893,43058,16077,52148,3809,16077,54846,15061,16077,52844,15576
402,NaN,NaN,879,17417,3874,19341,NaN,NaN,16056,22768,1130,16056,24248,4615,NaN,NaN,NaN


In [21]:
europe_german_afd=europe_german_afd.rename(columns={"Gültige Stimmen_2024":'Gültige_2024'})
europe_german_afd

,Nr_2024,Gebiet_2024,Alternative für Deutschland_2014,Gültige_2014,Alternative für Deutschland_2019,Gültige_2019,Alternative für Deutschland_2024,Gültige_2024,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017,Statistische Kennziffer_2021,Gültige_Zweitstimme_2021,AfD_Zweitstimme_2021
0,01001,"Flensburg, Stadt",1583,25174,2477,39410,4273,41552,01001,46265,1855,01001,49271,3702,01001,50818,2756
1,01002,"Kiel, Landeshauptstadt",4418,75336,6459,109018,9351,116547,01002,130686,4778,01002,137368,9524,01002,139961,7023
2,01003,"Lübeck, Hansestadt",4334,62509,6773,90485,10290,95954,01003,112029,4792,01003,117879,10461,01003,117674,7807
3,01004,"Neumünster, Stadt",1960,27250,2761,28805,5697,32737,01004,39388,1965,01004,39861,4124,01004,39971,3751
4,01051,Dithmarschen,2325,42350,5121,57769,11005,67527,01051,73649,3041,01051,77337,6626,01051,80616,7098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,16075,Saale-Orla-Kreis,3114,37348,11264,41630,14426,38324,16075,48979,3527,16075,50721,13452,16075,49552,15071
400,16076,Greiz,4559,47189,13297,52058,18399,52005,16076,61725,4714,16076,63402,16203,16076,61146,16908
401,16077,Altenburger Land,3023,37657,11371,41901,15893,43058,16077,52148,3809,16077,54846,15061,16077,52844,15576
402,NaN,NaN,879,17417,3874,19341,NaN,NaN,16056,22768,1130,16056,24248,4615,NaN,NaN,NaN


In [22]:
#and we calculate these percentage columns
perc_summed=europe_german_afd[europe_german_afd.columns[:2]]
for columnname in europe_german_afd.columns[2:]:
    if "Alternative" in columnname:
        perc_summed["eu_"+columnname[-4:]]=pd.to_numeric(europe_german_afd[columnname])*100/pd.to_numeric(europe_german_afd["Gültige_"+columnname[-4:]])
    if "AfD" in columnname:
        perc_summed["btw_"+columnname[-4:]]=pd.to_numeric(europe_german_afd[columnname])*100/pd.to_numeric(europe_german_afd["Gültige_Zweitstimme_"+columnname[-4:]])

europe_german_afd

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/2597229456.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_summed["eu_"+columnname[-4:]]=pd.to_numeric(europe_german_afd[columnname])*100/pd.to_numeric(europe_german_afd["Gültige_"+columnname[-4:]])
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/2597229456.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_summed["eu_"+columnname[-4:]]=pd.to_numeric(europe_german_afd[columnname])*100/pd.to_numeric(europe_german_a

,Nr_2024,Gebiet_2024,Alternative für Deutschland_2014,Gültige_2014,Alternative für Deutschland_2019,Gültige_2019,Alternative für Deutschland_2024,Gültige_2024,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017,Statistische Kennziffer_2021,Gültige_Zweitstimme_2021,AfD_Zweitstimme_2021
0,01001,"Flensburg, Stadt",1583,25174,2477,39410,4273,41552,01001,46265,1855,01001,49271,3702,01001,50818,2756
1,01002,"Kiel, Landeshauptstadt",4418,75336,6459,109018,9351,116547,01002,130686,4778,01002,137368,9524,01002,139961,7023
2,01003,"Lübeck, Hansestadt",4334,62509,6773,90485,10290,95954,01003,112029,4792,01003,117879,10461,01003,117674,7807
3,01004,"Neumünster, Stadt",1960,27250,2761,28805,5697,32737,01004,39388,1965,01004,39861,4124,01004,39971,3751
4,01051,Dithmarschen,2325,42350,5121,57769,11005,67527,01051,73649,3041,01051,77337,6626,01051,80616,7098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,16075,Saale-Orla-Kreis,3114,37348,11264,41630,14426,38324,16075,48979,3527,16075,50721,13452,16075,49552,15071
400,16076,Greiz,4559,47189,13297,52058,18399,52005,16076,61725,4714,16076,63402,16203,16076,61146,16908
401,16077,Altenburger Land,3023,37657,11371,41901,15893,43058,16077,52148,3809,16077,54846,15061,16077,52844,15576
402,NaN,NaN,879,17417,3874,19341,NaN,NaN,16056,22768,1130,16056,24248,4615,NaN,NaN,NaN


In [23]:
west_deutschland=europe_german_afd[pd.to_numeric(europe_german_afd["Nr_2024"].str[:2],errors="coerce")<11]
west_deutschland

,Nr_2024,Gebiet_2024,Alternative für Deutschland_2014,Gültige_2014,Alternative für Deutschland_2019,Gültige_2019,Alternative für Deutschland_2024,Gültige_2024,Statistische Kennziffer_2013,Gültige_Zweitstimme_2013,AfD_Zweitstimme_2013,Statistische Kennziffer_2017,Gültige_Zweitstimme_2017,AfD_Zweitstimme_2017,Statistische Kennziffer_2021,Gültige_Zweitstimme_2021,AfD_Zweitstimme_2021
0,01001,"Flensburg, Stadt",1583,25174,2477,39410,4273,41552,01001,46265,1855,01001,49271,3702,01001,50818,2756
1,01002,"Kiel, Landeshauptstadt",4418,75336,6459,109018,9351,116547,01002,130686,4778,01002,137368,9524,01002,139961,7023
2,01003,"Lübeck, Hansestadt",4334,62509,6773,90485,10290,95954,01003,112029,4792,01003,117879,10461,01003,117674,7807
3,01004,"Neumünster, Stadt",1960,27250,2761,28805,5697,32737,01004,39388,1965,01004,39861,4124,01004,39971,3751
4,01051,Dithmarschen,2325,42350,5121,57769,11005,67527,01051,73649,3041,01051,77337,6626,01051,80616,7098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,10042,Merzig-Wadern,2888,47360,4792,53919,8574,55666,10042,57967,2934,10042,61180,5667,10042,60816,5888
320,10043,Neunkirchen,3944,54607,7115,66848,12008,67102,10043,76301,4414,10043,78461,8556,10043,76088,8620
321,10044,Saarlouis,5532,82940,9825,101190,16273,102964,10044,112605,5958,10044,117718,11846,10044,116396,11865
322,10045,Saarpfalz-Kreis,4799,65332,8556,77321,12565,76985,10045,86143,4824,10045,88957,10044,10045,87833,9387


In [24]:
#we also quickly sum up the absolute gains of the AfD between 2024 and 2019
zugewinne_west_deutschland=(pd.to_numeric(west_deutschland["Alternative für Deutschland_2024"]).sum())-(pd.to_numeric(west_deutschland["Alternative für Deutschland_2019"]).sum())

In [25]:
zugewinne_west_deutschland

1532806

In [71]:
#and then we use the percentage wise dataframe and calculate relations
perc_summed_resorted=perc_summed[["Nr_2024","Gebiet_2024","btw_2013","eu_2014","btw_2017","eu_2019","btw_2021","eu_2024"]]
perc_summed_resorted["11years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["btw_2013"]
perc_summed_resorted["10years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["eu_2014"]

perc_summed_resorted["7years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["btw_2017"]
perc_summed_resorted["5years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["eu_2019"]
perc_summed_resorted["5years_perc"]=perc_summed_resorted["5years"]*100-100

perc_summed_resorted["3years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["btw_2021"]

perc_summed_resorted

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/582472399.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_summed_resorted["11years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["btw_2013"]
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/582472399.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_summed_resorted["10years"]=perc_summed_resorted["eu_2024"]/perc_summed_resorted["eu_2014"]
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/582

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years
0,01001,"Flensburg, Stadt",4.009510,6.288234,7.513548,6.285207,5.423275,10.283500,2.564777,1.635356,1.368661,1.636143,63.614349,1.896179
1,01002,"Kiel, Landeshauptstadt",3.656092,5.864394,6.933201,5.924710,5.017826,8.023373,2.194522,1.368150,1.157239,1.354222,35.422206,1.598974
2,01003,"Lübeck, Hansestadt",4.277464,6.933402,8.874354,7.485219,6.634431,10.723889,2.507067,1.546699,1.208413,1.432675,43.267541,1.616399
3,01004,"Neumünster, Stadt",4.988829,7.192661,10.345952,9.585141,9.384304,17.402328,3.488259,2.419456,1.682042,1.815553,81.555251,1.854408
4,01051,Dithmarschen,4.129045,5.489965,8.567697,8.864616,8.804704,16.297185,3.946963,2.968541,1.902166,1.838454,83.845356,1.850963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,16075,Saale-Orla-Kreis,7.201045,8.337796,26.521559,27.057411,30.414514,37.642209,5.227326,4.514647,1.419306,1.391198,39.119775,1.237640
400,16076,Greiz,7.637100,9.661150,25.555976,25.542664,27.651850,35.379290,4.632556,3.662017,1.384384,1.385106,38.510574,1.279455
401,16077,Altenburger Land,7.304211,8.027724,27.460526,27.137777,29.475437,36.910679,5.053342,4.597901,1.344136,1.360122,36.012166,1.252252
402,NaN,NaN,4.963106,5.046793,19.032498,20.029988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
perc_summed_resorted.sort_values(by=["11years"]).dropna().tail(50)

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years
356,14523,Vogtlandkreis,6.653499,10.172211,26.421082,23.878465,25.056007,34.105875,5.126006,3.352848,1.290858,1.428311,42.831102,1.361186
385,16061,Eichsfeld,5.214995,5.913818,18.182986,18.678781,19.683959,26.903442,5.158862,4.549251,1.479594,1.440321,44.032111,1.366770
256,09279,Dingolfing-Landau,3.597576,6.485380,18.299781,11.327095,13.673836,18.791576,5.223399,2.897529,1.026874,1.658993,65.899336,1.374272
399,16075,Saale-Orla-Kreis,7.201045,8.337796,26.521559,27.057411,30.414514,37.642209,5.227326,4.514647,1.419306,1.391198,39.119775,1.237640
351,13075,Vorpommern-Greifswald,6.401912,7.842417,22.715704,21.747770,21.978553,33.623840,5.252156,4.287434,1.480202,1.546082,54.608220,1.529848
361,14627,Meißen,7.119960,12.159943,32.876558,31.040820,29.770858,37.689096,5.293442,3.099447,1.146382,1.214179,21.417850,1.265973
265,09376,Schwandorf,3.330487,7.336153,16.805760,11.680544,13.219614,17.869863,5.365541,2.435863,1.063318,1.529883,52.988281,1.351769
342,12071,Spree-Neiße,6.972134,9.546176,28.884640,30.880025,27.394695,37.844710,5.427995,3.964385,1.310202,1.225540,22.554016,1.381461
50,03452,Aurich,3.145421,4.428735,8.924106,9.098699,8.226367,17.087931,5.432638,3.858422,1.914806,1.878063,87.806301,2.077215
107,05915,"Hamm, Stadt",3.073981,4.469575,11.137809,10.902548,9.236656,16.778533,5.458241,3.753943,1.506448,1.538955,53.895517,1.816516


In [73]:
perc_summed_resorted.sort_values(by=["5years"]).dropna()

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years
162,07315,"Mainz, kreisfreie Stadt",4.314629,6.000879,7.331953,5.876848,4.351049,6.219183,1.441418,1.036379,0.848230,1.058251,5.825143,1.429353
118,06413,"Offenbach am Main, Stadt",5.636248,9.559543,12.031549,10.113561,7.991279,11.161096,1.980235,1.167534,0.927652,1.103577,10.357731,1.396660
203,08311,"Freiburg im Breisgau, Stadtkreis",3.693911,5.695366,7.246919,5.299810,4.353852,5.885669,1.593344,1.033414,0.812162,1.110543,11.054338,1.351830
223,09162,"München, Landeshauptstadt",4.497582,7.792685,8.365956,5.984344,4.543810,6.662299,1.481307,0.854943,0.796358,1.113288,11.328820,1.466236
381,16052,"Gera, Stadt",7.769928,11.162000,28.517740,29.615636,27.231860,32.970755,4.243380,2.953839,1.156149,1.113289,11.328879,1.210742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,03255,Holzminden,3.474783,4.739987,9.620642,8.327548,9.036478,16.640340,4.788886,3.510630,1.729650,1.998228,99.822802,1.841463
54,03456,Grafschaft Bentheim,2.535045,2.760817,5.253278,4.726121,5.159982,9.573557,3.776484,3.467653,1.822397,2.025669,102.566914,1.855347
57,03459,Osnabrück,2.744091,3.598509,7.034149,5.620668,6.211628,11.457545,4.175351,3.183970,1.628846,2.038467,103.846669,1.844532
52,03454,Emsland,2.499306,3.072953,6.300585,5.442833,6.093659,11.646932,4.660066,3.790144,1.848548,2.139866,113.986563,1.911320


In [74]:
perc_summed_resorted.to_csv("landkreise_all.csv",index=False)

In [75]:
perc_summed_resorted[perc_summed_resorted["Nr_2024"]=="03159"]

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years
25,03159,Göttingen,3.71789,5.021836,8.098812,6.784111,6.349654,10.797233,2.904129,2.150057,1.333187,1.591547,59.154709,1.700444


In [76]:
threeyears=perc_summed_resorted.sort_values(by=["3years"])
threeyears[threeyears["Gebiet_2024"].astype(str).str.contains("Berlin")]

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years
324,11000,"Berlin, Stadt",4.925825,7.912537,12.047348,9.896594,8.4143,11.604883,2.355927,1.466645,0.963273,1.172614,17.261388,1.379186


In [77]:
threeyears.to_csv("afd_three_years.csv",index=False)
threeyears[threeyears["Nr_2024"]=="03159"]

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years
25,03159,Göttingen,3.71789,5.021836,8.098812,6.784111,6.349654,10.797233,2.904129,2.150057,1.333187,1.591547,59.154709,1.700444


In [78]:
#then for correlation we add the data on the quota of people working in agriculture
landwirtschaft=pd.read_csv("../data/landwirtschaft.csv",sep=";")
landwirtschaft=landwirtschaft.rename(columns={"Unnamed: 2":"Landwirtschaft"})
ags_filtered=landwirtschaft[landwirtschaft["Unnamed: 0"].str.len()==5]
ags_filtered["Landwirtschaft"]=pd.to_numeric(ags_filtered["Landwirtschaft"])
ags_filtered_landwirtschaft=ags_filtered[ags_filtered["Landwirtschaft"]<100]
ags_filtered_landwirtschaft["ags_number"]=pd.to_numeric(ags_filtered_landwirtschaft["Unnamed: 0"])
landwirtschaft_west=ags_filtered_landwirtschaft[ags_filtered_landwirtschaft["ags_number"]<11000]

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/3714475588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ags_filtered["Landwirtschaft"]=pd.to_numeric(ags_filtered["Landwirtschaft"])
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/3714475588.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ags_filtered_landwirtschaft["ags_number"]=pd.to_numeric(ags_filtered_landwirtschaft["Unnamed: 0"])


In [135]:
landwirtschaft_west.to_csv("landwirtschaft_west.csv",index=False)
landwirtschaft_west

,Unnamed: 0,"Anteil ET Land- u. Forstwirtschaft, Fischerei (%)",Landwirtschaft,ags_number,bl
10,01002,Kiel,0.1,1002,01
11,01003,Lübeck,0.2,1003,01
12,01004,Neumünster,0.7,1004,01
13,01051,Dithmarschen,5.5,1051,01
14,01053,Herzogtum Lauenburg,2.6,1053,01
...,...,...,...,...,...
328,10042,Merzig-Wadern,1.4,10042,10
329,10043,Neunkirchen,0.4,10043,10
330,10044,Saarlouis,0.5,10044,10
331,10045,Saarpfalz-Kreis,0.4,10045,10


In [80]:
#and then we also divide into spd or green reigned districts and cdu reigned districts
landwirtschaft_west["bl"]=landwirtschaft_west["Unnamed: 0"].str[:2]
searchfor = ['03',"07","08",'10']
spd_west=landwirtschaft_west[landwirtschaft_west.bl.str.contains('|'.join(searchfor))]
cdu_west=landwirtschaft_west[~landwirtschaft_west.bl.str.contains('|'.join(searchfor))]

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_7594/1875603708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  landwirtschaft_west["bl"]=landwirtschaft_west["Unnamed: 0"].str[:2]


In [81]:
landwirtschaft_west_merged=pd.merge(perc_summed_resorted,landwirtschaft_west,left_on="Nr_2024",right_on="Unnamed: 0")
#landwirtschaft_west_merged_2=pd.merge(ags_filtered_schweine_rinder,landwirtschaft_west_merged,right_on="Nr_2024",left_on="Unnamed: 1")
landwirtschaft_west_merged=landwirtschaft_west_merged.rename(columns={"Unnamed: 2":"Landwirtschaft"})

In [82]:
landwirtschaft_spd_west_merged=pd.merge(perc_summed_resorted,spd_west,left_on="Nr_2024",right_on="Unnamed: 0")
landwirtschaft_cdu_west_merged=pd.merge(perc_summed_resorted,cdu_west,left_on="Nr_2024",right_on="Unnamed: 0")

landwirtschaft_spd_west_merged

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years,Unnamed: 0,"Anteil ET Land- u. Forstwirtschaft, Fischerei (%)",Landwirtschaft,ags_number,bl
0,03101,"Braunschweig, Stadt",3.578733,5.626977,8.365097,6.848268,5.914110,9.606767,2.684405,1.707270,1.148435,1.402803,40.280254,1.624381,03101,Braunschweig,0.1,3101,03
1,03102,"Salzgitter, Stadt",3.917159,5.789967,16.437597,14.988322,13.472299,22.032575,5.624632,3.805302,1.340377,1.469983,46.998277,1.635398,03102,Salzgitter,0.4,3102,03
2,03103,"Wolfsburg, Stadt",3.809301,6.595599,11.681540,10.196541,9.314725,15.722671,4.127443,2.383812,1.345942,1.541961,54.196132,1.687937,03103,Wolfsburg,0.1,3103,03
3,03151,Gifhorn,3.560758,4.870405,11.420632,9.780852,10.481903,17.110527,4.805304,3.513163,1.498212,1.749390,74.939028,1.632387,03151,Gifhorn,3.0,3151,03
4,03153,Goslar,4.133062,6.797389,11.136709,10.272480,9.236021,16.309755,3.946168,2.399415,1.464504,1.587714,58.771354,1.765885,03153,Goslar,1.0,3153,03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,10042,Merzig-Wadern,5.061501,6.097973,9.262831,8.887405,9.681663,15.402580,3.043086,2.525852,1.662837,1.733079,73.307949,1.590902,10042,Merzig-Wadern,1.4,10042,10
125,10043,Neunkirchen,5.784983,7.222517,10.904781,10.643550,11.328987,17.895145,3.093379,2.477688,1.641037,1.681314,68.131361,1.579589,10043,Neunkirchen,0.4,10043,10
126,10044,Saarlouis,5.291062,6.669882,10.063032,9.709457,10.193649,15.804553,2.987029,2.369540,1.570556,1.627748,62.774832,1.550431,10044,Saarlouis,0.5,10044,10
127,10045,Saarpfalz-Kreis,5.599991,7.345558,11.290848,11.065558,10.687327,16.321361,2.914534,2.221936,1.445539,1.474970,47.496959,1.527170,10045,Saarpfalz-Kreis,0.4,10045,10


In [83]:
!pip install scipy

You should consider upgrading via the '/Users/maier-borst.h/Documents/CodeRelated/HBDataviz/2024/23_EuropawahlAfD/Wrangle/bin/python3 -m pip install --upgrade pip' command.


In [84]:
import scipy

In [85]:
from scipy import stats
res = stats.spearmanr(landwirtschaft_west_merged["Landwirtschaft"], landwirtschaft_west_merged["3years"])
res

SignificanceResult(statistic=0.07506196548644933, pvalue=0.18321496265163148)

In [86]:
res = stats.spearmanr(landwirtschaft_west_merged["Landwirtschaft"], landwirtschaft_west_merged["5years"])
res

SignificanceResult(statistic=0.5487335621008245, pvalue=3.04063350580699e-26)

In [87]:
res = stats.spearmanr(landwirtschaft_west_merged["Landwirtschaft"], landwirtschaft_west_merged["7years"])
res

SignificanceResult(statistic=0.27877926949661447, pvalue=4.7475732503974817e-07)

In [88]:
res = stats.spearmanr(landwirtschaft_west_merged["Landwirtschaft"], landwirtschaft_west_merged["10years"])
res

SignificanceResult(statistic=0.31326917433593976, pvalue=1.2659343356274726e-08)

In [89]:
res = stats.spearmanr(landwirtschaft_west_merged["Landwirtschaft"], landwirtschaft_west_merged["11years"])
res

SignificanceResult(statistic=0.35845332241072636, pvalue=5.160408241438454e-11)

In [90]:
res = stats.spearmanr(landwirtschaft_spd_west_merged["Landwirtschaft"], landwirtschaft_spd_west_merged["3years"])
res

SignificanceResult(statistic=0.41230990518842126, pvalue=1.203108027270676e-06)

In [91]:
res = stats.spearmanr(landwirtschaft_spd_west_merged["Landwirtschaft"], landwirtschaft_spd_west_merged["5years"])
res

SignificanceResult(statistic=0.4352352778792644, pvalue=2.5426144672820916e-07)

In [92]:
res = stats.spearmanr(landwirtschaft_spd_west_merged["Landwirtschaft"], landwirtschaft_spd_west_merged["7years"])
res

SignificanceResult(statistic=0.43357397845528484, pvalue=2.856840865171681e-07)

In [93]:
res = stats.spearmanr(landwirtschaft_spd_west_merged["Landwirtschaft"], landwirtschaft_spd_west_merged["10years"])
res

SignificanceResult(statistic=0.3830609668450226, pvalue=7.446462378606699e-06)

In [94]:
res = stats.spearmanr(landwirtschaft_spd_west_merged["Landwirtschaft"], landwirtschaft_spd_west_merged["11years"])
res

SignificanceResult(statistic=0.3425800781873107, pvalue=7.066640119352465e-05)

In [95]:
res = stats.spearmanr(landwirtschaft_west_merged["Landwirtschaft"], landwirtschaft_west_merged["btw_2021"]/landwirtschaft_west_merged["eu_2019"])
res

SignificanceResult(statistic=0.43804993320333746, pvalue=3.0133794142248864e-16)

In [96]:
landwirtschaft_west_merged.to_csv("corr_base.csv",index=False)

In [97]:
res = stats.spearmanr(landwirtschaft_spd_west_merged["Landwirtschaft"], landwirtschaft_spd_west_merged["3years"])
res

SignificanceResult(statistic=0.41230990518842126, pvalue=1.203108027270676e-06)

In [98]:
res = stats.spearmanr(landwirtschaft_cdu_west_merged["Landwirtschaft"], landwirtschaft_cdu_west_merged["3years"])
res

SignificanceResult(statistic=-0.1473012506783843, pvalue=0.04424148233073329)

In [99]:
landwirtschaft_west_merged.sort_values(by="5years",ascending=False).head(20)

,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years,Unnamed: 0,"Anteil ET Land- u. Forstwirtschaft, Fischerei (%)",Landwirtschaft,ags_number,bl
50,03453,Cloppenburg,2.442619,3.701259,9.060422,7.458055,9.549417,16.635052,6.810333,4.494430,1.836013,2.230481,123.048123,1.741997,03453,Cloppenburg,7.2,3453,03
51,03454,Emsland,2.499306,3.072953,6.300585,5.442833,6.093659,11.646932,4.660066,3.790144,1.848548,2.139866,113.986563,1.911320,03454,Emsland,4.1,3454,03
56,03459,Osnabrück,2.744091,3.598509,7.034149,5.620668,6.211628,11.457545,4.175351,3.183970,1.628846,2.038467,103.846669,1.844532,03459,"Osnabrück, Landkreis",3.3,3459,03
53,03456,Grafschaft Bentheim,2.535045,2.760817,5.253278,4.726121,5.159982,9.573557,3.776484,3.467653,1.822397,2.025669,102.566914,1.855347,03456,Grafschaft Bentheim,3.4,3456,03
29,03255,Holzminden,3.474783,4.739987,9.620642,8.327548,9.036478,16.640340,4.788886,3.510630,1.729650,1.998228,99.822802,1.841463,03255,Holzminden,1.6,3255,03
10,01059,Schleswig-Flensburg,4.116428,6.519981,6.560556,6.011148,5.915951,11.724074,2.848118,1.798176,1.787055,1.950389,95.038866,1.981773,01059,Schleswig-Flensburg,4.3,1059,01
12,01061,Steinburg,4.447830,5.823809,8.232852,7.776373,7.979447,15.132541,3.402230,2.598392,1.838068,1.945964,94.596382,1.896440,01061,Steinburg,4.1,1061,01
54,03457,Leer,3.399653,5.292650,9.220794,9.038149,8.851753,17.367766,5.108688,3.281488,1.883543,1.921606,92.160647,1.962071,03457,Leer,2.8,3457,03
289,09673,Rhön-Grabfeld,3.224178,5.168836,10.377151,7.560083,10.057876,14.438365,4.478154,2.793349,1.391361,1.909816,90.981575,1.435528,09673,Rhön-Grabfeld,1.8,9673,09
5,01054,Nordfriesland,3.780780,5.679142,5.805914,5.665315,5.233519,10.759486,2.845838,1.894562,1.853194,1.899186,89.918588,2.055880,01054,Nordfriesland,3.5,1054,01


In [136]:
landwirtschaft_west_merged.to_csv("landwirtschaft_afd_merge.csv",index=False)

In [100]:
landwirtschaft_west_merged.head(20).to_csv("top20_zuwaechse.csv",index=False)

In [154]:
landwirtschaft_cdu_west_merged["3years_perc"]=landwirtschaft_cdu_west_merged["3years"]*100-100
landwirtschaft_cdu_west_merged.to_csv("corr_cdu.csv",index=False)

In [155]:
landwirtschaft_spd_west_merged["3years_perc"]=landwirtschaft_spd_west_merged["3years"]*100-100

landwirtschaft_spd_west_merged.to_csv("corr_spd.csv",index=False)

In [130]:
above1_9=landwirtschaft_west_merged.sort_values(by="5years",ascending=False).head(10)
above1_9_T=above1_9[["Gebiet_2024","Landwirtschaft"]].T
above1_9_T.columns=above1_9_T.iloc[0]
above1_9_T=above1_9_T.iloc[1:]
above1_9_T

Gebiet_2024,Cloppenburg,Emsland,Osnabrück,Grafschaft Bentheim,Holzminden,Schleswig-Flensburg,Steinburg,Leer,Rhön-Grabfeld,Nordfriesland
Landwirtschaft,7.2,4.1,3.3,3.4,1.6,4.3,4.1,2.8,1.8,3.5


In [131]:
below1_2=landwirtschaft_west_merged.sort_values(by="5years",ascending=False).tail(10)

below1_2_T=below1_2[["Gebiet_2024","Landwirtschaft"]].T
below1_2_T.columns=below1_2_T.iloc[0]
below1_2_T=below1_2_T.iloc[1:]
below1_2_T

Gebiet_2024,"Köln, Stadt",Hochtaunuskreis,"Stuttgart, Stadtkreis",Main-Taunus-Kreis,"Frankfurt am Main, Stadt","Heidelberg, Stadtkreis","Bonn, Stadt","München, Landeshauptstadt","Freiburg im Breisgau, Stadtkreis","Mainz, kreisfreie Stadt"
Landwirtschaft,0.1,0.4,0.2,0.4,0.0,0.2,0.0,0.1,0.3,0.2


In [134]:
above1_9_T["group"]="Stärksten Zugewinne für die AfD"
below1_2_T["group"]="Geringsten Zugewinne für die AfD"
comparison_lw_zuwachs=pd.concat([above1_9_T,below1_2_T],axis=0)
comparison_lw_zuwachs.to_csv("top_flop.csv",index=False)
comparison_lw_zuwachs

Gebiet_2024,Cloppenburg,Emsland,Osnabrück,Grafschaft Bentheim,Holzminden,Schleswig-Flensburg,Steinburg,Leer,Rhön-Grabfeld,Nordfriesland,...,"Köln, Stadt",Hochtaunuskreis,"Stuttgart, Stadtkreis",Main-Taunus-Kreis,"Frankfurt am Main, Stadt","Heidelberg, Stadtkreis","Bonn, Stadt","München, Landeshauptstadt","Freiburg im Breisgau, Stadtkreis","Mainz, kreisfreie Stadt"
Landwirtschaft,7.2,4.1,3.3,3.4,1.6,4.3,4.1,2.8,1.8,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Landwirtschaft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1,0.4,0.2,0.4,0.0,0.2,0.0,0.1,0.3,0.2


In [143]:
landwirtschaft_cdu_west_merged.sort_values(by="3years",ascending=False).tail(10).describe()

,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,7years,5years,5years_perc,3years,Landwirtschaft,ags_number
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,4.354990,8.544131,15.863580,10.578470,11.647607,15.474442,3.786986,1.901099,0.969354,1.449779,44.977884,1.329858,1.660000,9350.000000
std,0.922122,1.674047,3.096055,1.816991,2.897502,3.747695,1.456823,0.664517,0.097017,0.153738,15.373765,0.030367,1.128618,189.245872
min,3.205563,6.854709,11.436498,7.729444,7.768681,9.929718,1.997883,1.065670,0.801159,1.242517,24.251745,1.276725,0.200000,9161.000000
25%,3.637810,7.107104,14.328340,9.624598,9.037805,12.203410,2.376378,1.382966,0.895498,1.313542,31.354237,1.317295,0.475000,9197.750000
50%,4.180948,8.003348,15.401733,11.052861,11.846215,15.988216,3.912420,1.886871,0.980754,1.492738,49.273759,1.344087,2.100000,9324.000000
75%,5.136645,10.139084,17.708611,11.626333,13.651127,18.382598,5.088474,2.487670,1.039146,1.527204,52.720378,1.349892,2.475000,9439.750000
max,5.682279,10.887149,20.679722,13.343730,15.850876,20.237204,5.787910,2.688455,1.108509,1.722251,72.225080,1.353421,3.000000,9775.000000


In [152]:
landwirtschaft_cdu_west_merged_top=landwirtschaft_cdu_west_merged.sort_values(by="3years",ascending=False).head(10)

landwirtschaft_cdu_west_merged_flop=landwirtschaft_cdu_west_merged.sort_values(by="3years",ascending=False).tail(10)
landwirtschaft_cdu_west_merged_top["group"]="Stärksten Zugewinne für die AfD"
landwirtschaft_cdu_west_merged_flop["group"]="Geringsten Zugewinne für die AfD"
comparison_lw_zuwachs=pd.concat([landwirtschaft_cdu_west_merged_top,landwirtschaft_cdu_west_merged_flop],axis=0)
comparison_lw_zuwachs.to_csv("top_flop_cdu.csv",index=False)
comparison_lw_zuwachs


,Nr_2024,Gebiet_2024,btw_2013,eu_2014,btw_2017,eu_2019,btw_2021,eu_2024,11years,10years,...,5years,5years_perc,3years,Unnamed: 0,"Anteil ET Land- u. Forstwirtschaft, Fischerei (%)",Landwirtschaft,ags_number,bl,3years_perc,group
5,01054,Nordfriesland,3.780780,5.679142,5.805914,5.665315,5.233519,10.759486,2.845838,1.894562,...,1.899186,89.918588,2.055880,01054,Nordfriesland,3.5,1054,01,105.587969,Stärksten Zugewinne für die AfD
10,01059,Schleswig-Flensburg,4.116428,6.519981,6.560556,6.011148,5.915951,11.724074,2.848118,1.798176,...,1.950389,95.038866,1.981773,01059,Schleswig-Flensburg,4.3,1059,01,98.177328,Stärksten Zugewinne für die AfD
44,05554,Borken,2.765581,3.397577,6.137097,5.418706,5.062349,10.015645,3.621534,2.947879,...,1.848346,84.834649,1.978458,05554,Borken,2.2,5554,05,97.845792,Stärksten Zugewinne für die AfD
25,05154,Kleve,3.933566,4.561281,6.931372,6.402598,5.673659,11.140568,2.832180,2.442421,...,1.740007,74.000736,1.963560,05154,Kleve,5.4,5154,05,96.355988,Stärksten Zugewinne für die AfD
45,05558,Coesfeld,2.847724,3.710790,6.218353,5.454500,4.732470,9.273061,3.256306,2.498945,...,1.700075,70.007543,1.959455,05558,Coesfeld,3.3,5558,05,95.945480,Stärksten Zugewinne für die AfD
28,05166,Viersen,4.105020,5.188295,7.464916,7.032552,5.960795,11.450445,2.789376,2.206976,...,1.628206,62.820621,1.920959,05166,Viersen,2.4,5166,05,92.095949,Stärksten Zugewinne für die AfD
12,01061,Steinburg,4.447830,5.823809,8.232852,7.776373,7.979447,15.132541,3.402230,2.598392,...,1.945964,94.596382,1.896440,01061,Steinburg,4.1,1061,01,89.643991,Stärksten Zugewinne für die AfD
47,05566,Steinfurt,2.763236,3.251326,6.491264,5.609079,5.503155,10.396993,3.762615,3.197770,...,1.853601,85.360067,1.889279,05566,Steinfurt,1.8,5566,05,88.927858,Stärksten Zugewinne für die AfD
33,05334,Städteregion Aachen,3.210131,4.009470,7.425628,6.900679,5.783773,10.749029,3.348471,2.680910,...,1.557677,55.767685,1.858480,05334,Städteregion Aachen,0.3,5334,05,85.848028,Stärksten Zugewinne für die AfD
2,01004,"Neumünster, Stadt",4.988829,7.192661,10.345952,9.585141,9.384304,17.402328,3.488259,2.419456,...,1.815553,81.555251,1.854408,01004,Neumünster,0.7,1004,01,85.440799,Stärksten Zugewinne für die AfD


In [160]:
landwirtschaft_spd_west_merged_top=landwirtschaft_spd_west_merged.sort_values(by="3years",ascending=False).head(10)
landwirtschaft_spd_west_merged_top=landwirtschaft_spd_west_merged_top[["Gebiet_2024","Landwirtschaft"]].T
landwirtschaft_spd_west_merged_top.columns=landwirtschaft_spd_west_merged_top.iloc[0]
landwirtschaft_spd_west_merged_top=landwirtschaft_spd_west_merged_top.iloc[1:]
landwirtschaft_spd_west_merged_top

Gebiet_2024,Aurich,"Emden, Stadt",Leer,Wittmund,Emsland,Ammerland,Wesermarsch,Vechta,Rotenburg (Wümme),Friesland
Landwirtschaft,3.1,0.3,2.8,4.1,4.1,5.9,3.2,6.0,4.5,3.0


In [161]:

landwirtschaft_spd_west_merged_flop=landwirtschaft_spd_west_merged.sort_values(by="3years",ascending=False).tail(10)
landwirtschaft_spd_west_merged_flop=landwirtschaft_spd_west_merged_flop[["Gebiet_2024","Landwirtschaft"]].T
landwirtschaft_spd_west_merged_flop.columns=landwirtschaft_spd_west_merged_flop.iloc[0]
landwirtschaft_spd_west_merged_flop=landwirtschaft_spd_west_merged_flop.iloc[1:]
landwirtschaft_spd_west_merged_flop

Gebiet_2024,Enzkreis,"Pforzheim, Stadtkreis","Heidelberg, Stadtkreis",Schwäbisch Hall,"Mainz, kreisfreie Stadt",Rottweil,Hohenlohekreis,"Ulm, Stadtkreis",Main-Tauber-Kreis,"Freiburg im Breisgau, Stadtkreis"
Landwirtschaft,0.7,0.1,0.2,1.8,0.2,1.0,2.6,0.2,1.7,0.3


In [162]:


landwirtschaft_spd_west_merged_top["group"]="Stärksten Zugewinne für die AfD"
landwirtschaft_spd_west_merged_flop["group"]="Geringsten Zugewinne für die AfD"
comparison_lw_zuwachs=pd.concat([landwirtschaft_spd_west_merged_top,landwirtschaft_spd_west_merged_flop],axis=0)
comparison_lw_zuwachs.to_csv("top_flop_spd.csv",index=False)
comparison_lw_zuwachs


Gebiet_2024,Aurich,"Emden, Stadt",Leer,Wittmund,Emsland,Ammerland,Wesermarsch,Vechta,Rotenburg (Wümme),Friesland,...,Enzkreis,"Pforzheim, Stadtkreis","Heidelberg, Stadtkreis",Schwäbisch Hall,"Mainz, kreisfreie Stadt",Rottweil,Hohenlohekreis,"Ulm, Stadtkreis",Main-Tauber-Kreis,"Freiburg im Breisgau, Stadtkreis"
Landwirtschaft,3.1,0.3,2.8,4.1,4.1,5.9,3.2,6.0,4.5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Landwirtschaft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,0.1,0.2,1.8,0.2,1.0,2.6,0.2,1.7,0.3


In [167]:
landwirtschaft_cdu_west_merged_top=landwirtschaft_cdu_west_merged.sort_values(by="3years",ascending=False).head(10)
landwirtschaft_cdu_west_merged_top=landwirtschaft_cdu_west_merged_top[["Gebiet_2024","Landwirtschaft"]].T
landwirtschaft_cdu_west_merged_top.columns=landwirtschaft_cdu_west_merged_top.iloc[0]
landwirtschaft_cdu_west_merged_top=landwirtschaft_cdu_west_merged_top.iloc[1:]
landwirtschaft_cdu_west_merged_top

Gebiet_2024,Nordfriesland,Schleswig-Flensburg,Borken,Kleve,Coesfeld,Viersen,Steinburg,Steinfurt,Städteregion Aachen,"Neumünster, Stadt"
Landwirtschaft,3.5,4.3,2.2,5.4,3.3,2.4,4.1,1.8,0.3,0.7


In [168]:

landwirtschaft_cdu_west_merged_flop=landwirtschaft_cdu_west_merged.sort_values(by="3years",ascending=False).tail(10)
landwirtschaft_cdu_west_merged_flop=landwirtschaft_cdu_west_merged_flop[["Gebiet_2024","Landwirtschaft"]].T
landwirtschaft_cdu_west_merged_flop.columns=landwirtschaft_cdu_west_merged_flop.iloc[0]
landwirtschaft_cdu_west_merged_flop=landwirtschaft_cdu_west_merged_flop.iloc[1:]
landwirtschaft_cdu_west_merged_flop

Gebiet_2024,"Rosenheim, kreisfreie Stadt",Schwandorf,Bamberg,Neu-Ulm,Cham,Bad Tölz-Wolfratshausen,"Ingolstadt, kreisfreie Stadt",Freyung-Grafenau,"Bamberg, kreisfreie Stadt",Regen
Landwirtschaft,0.2,2.5,1.8,1.0,3.0,2.4,0.2,2.8,0.3,2.4


In [169]:


landwirtschaft_cdu_west_merged_top["group"]="Stärksten Zugewinne für die AfD"
landwirtschaft_cdu_west_merged_flop["group"]="Geringsten Zugewinne für die AfD"
comparison_lw_zuwachs=pd.concat([landwirtschaft_cdu_west_merged_top,landwirtschaft_cdu_west_merged_flop],axis=0)
comparison_lw_zuwachs.to_csv("top_flop_cdu.csv",index=False)
comparison_lw_zuwachs


Gebiet_2024,Nordfriesland,Schleswig-Flensburg,Borken,Kleve,Coesfeld,Viersen,Steinburg,Steinfurt,Städteregion Aachen,"Neumünster, Stadt",...,"Rosenheim, kreisfreie Stadt",Schwandorf,Bamberg,Neu-Ulm,Cham,Bad Tölz-Wolfratshausen,"Ingolstadt, kreisfreie Stadt",Freyung-Grafenau,"Bamberg, kreisfreie Stadt",Regen
Landwirtschaft,3.5,4.3,2.2,5.4,3.3,2.4,4.1,1.8,0.3,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Landwirtschaft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,2.5,1.8,1.0,3.0,2.4,0.2,2.8,0.3,2.4
